## Utiliser un travail de balayage pour le réglage des hyperparamètres


### Créer un script d’apprentissage pour le réglages des hyperparamètres
Pour exécuter un travail de balayage, vous devez créer un script d’apprentissage comme pour tout autre travail d’apprentissage. Toutefois, votre script doit :

Inclure un argument pour chaque hyperparamètre que vous souhaitez faire varier.
Consigner la mesure de performance cible avec MLflow. Une métrique journalisée permet au travail de balayage d’évaluer les performances des essais initiés et d’identifier celui qui produit le modèle le plus performant

Par exemple, l’exemple de script suivant forme un modèle de régression logistique à l’aide d’un argument --regularization pour définir l’hyperparamètre de taux de régularisation et enregistre la mesure de précision avec le nom Accuracy :

```python
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import mlflow

# get regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01)
args = parser.parse_args()
reg = args.reg_rate

# load the training dataset
data = pd.read_csv("data.csv")

# separate features and labels, and split for training/validatiom
X = data[['feature1','feature2','feature3','feature4']].values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# train a logistic regression model with the reg hyperparameter
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate and log accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
mlflow.log_metric("Accuracy", acc)
```


### Configurer et exécuter un travail de balayage
Pour préparer le travail de balayage, vous devez d’abord créer un travail de commande de base qui spécifie le script à exécuter et définit les paramètres utilisés par le script :

```python
from azure.ai.ml import command

# configure command job as base
job = command(
    code="./src",
    command="python train.py --regularization ${{inputs.regularization}}",
    inputs={
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    )
```


Vous pouvez ensuite écraser vos paramètres d’entrée par votre espace de recherche :
```python
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)
```


Enfin, appelez sweep() sur votre travail de commande pour parcourir votre espace de recherche :

In [ ]:
from azure.ai.ml import MLClient

# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="Accuracy",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-example"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
